## Remove red stamp in image

In [3]:
def apply_brightness_contrast(input_img, brightness = 255, contrast = 125):
    def map(x, in_min, in_max, out_min, out_max):
        return int((x-in_min) * (out_max-out_min) / (in_max-in_min) + out_min)

    brightness = map(brightness, 0, 510, -255, 255)
    contrast = map(contrast, 0, 254, -127, 127)
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    if contrast != 0:
        f = float(131 * (contrast + 127)) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)
    return buf


def qualify_bounding_boxes(bboxes):
    result_bboxes = list()
    for bbox in bboxes:
        if (bbox[0]* bbox[1] < 250):
            continue
        result_bboxes.append(bbox)
    return result_bboxes


In [65]:
import cv2
import numpy as np
from PIL import ImageEnhance, Image


def imshow(img, figsize=(25, 25), **kwargs):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    ax.imshow(img, **kwargs)


def is_rec_in_rec(recA, recB):
    return recA[0] < recB[0] < recB[2] < recA[2] and recA[1] < recB[1] < recB[3] < recA[3]

def check_rec_in_rec(all_rects):
    boolean_list = list()
    for rect in all_rects:
       boolean_list.append(True if True in [is_rec_in_rec(rect, that_rect) for that_rect in all_rects] else False)
    return boolean_list



def remove_redstamp(img_path):

    def is_rec_in_rec(recA, recB):
        return recA[0] < recB[0] < recB[2]-recB[0] < recA[2]-recA[0] and recA[1] < recB[1] < recB[3]-recB[1] < recA[3]-recA[1]

    def check_rec_in_rec(this_rect, all_rects):
        return True if True in [is_rec_in_rec(this_rect, that_rect) for that_rect in all_rects] else False

        
    img = cv2.imread(img_path)
    copied_image = img.copy()

    img = apply_brightness_contrast(img, brightness = 255, contrast = 145)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower_red_mask_1 = np.array([0,25,0], np.uint8)
    upper_red_mask_1 = np.array([10,255,255], np.uint8)
    mask1 = cv2.inRange(img_hsv, lower_red_mask_1, upper_red_mask_1)

    output_mask1 = cv2.bitwise_and(copied_image, copied_image, mask=mask1)

    lower_red_mask_2 = np.array([160,25,0])
    upper_red_mask_2 = np.array([180,255,255])
    mask2 = cv2.inRange(img_hsv, lower_red_mask_2, upper_red_mask_2)

    output_mask2 = cv2.bitwise_and(copied_image, copied_image, mask=mask2)

    output_mask = cv2.add(output_mask1, output_mask2) 
    result_image = copied_image - output_mask

    _, thresh = cv2.threshold(result_image, 5, 255, cv2.THRESH_BINARY_INV)
    opening_thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT,(11, 11)))
    opening_thresh_ = cv2.cvtColor(opening_thresh, cv2.COLOR_BGR2GRAY)

    contours = cv2.findContours(opening_thresh_, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    result_bboxes = list()
    for cntr in contours:
        x, y, w, h = cv2.boundingRect(cntr)
        if (w >= 224 or h >= 224) and not check_rec_in_rec([x, y, w, h], result_bboxes):
            result_bboxes.append([x, y, w, h])

    return result_bboxes

In [68]:
import glob


remove_redstamp("C:/Users/ASUS/PROJECT001/data/test-repo-master/sec/sec7.jpg")



[[298, 479, 289, 294],
 [628, 261, 395, 167],
 [0, 247, 144, 285],
 [315, 0, 416, 140]]

## By Deep learning method

In [1]:
%cd C:/Users/ASUS/PROJECT001/stamp_processing

from stamp_processing import StampRemover

remover = StampRemover(detection_weight=None, removal_weight=r"C:\Users\ASUS\PROJECT001\unet_removal_weight.pkl")

C:\Users\ASUS\PROJECT001\stamp_processing


c:\Users\ASUS\anaconda3_2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from fastai.vision.all import *

unet_model = load_learner(r"C:\Users\ASUS\PROJECT001\unet_removal_weight.pkl").model
unet_model

CustomDynamicUnet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps

In [ ]:
import cv2

preds = remover([cv2.imread("C:/Users/ASUS/PROJECT001/data/test-repo-master/sec/sec5.jpg")])

def imshow(img, figsize=(25, 25), **kwargs):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    ax.imshow(img, **kwargs)

imshow(preds[0])

## Convert Model to ONNX
### Convert FastAi Model to Torch.Jit.Script

In [70]:
!pip3 install onnxruntime

In [77]:
%cd C:/Users/ASUS/PROJECT001/stamp_processing

import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

from PIL import Image
from fastai.vision.all import *

unet_model = load_learner(r"C:\Users\ASUS\PROJECT001\unet_removal_weight.pkl").model.cuda()

dummy_input = torch.randn([1,3, 320, 320], requires_grad=True).cuda()

torch.jit.save(torch.jit.trace(unet_model, dummy_input), 'RedStampRemover160x480.pt')

C:\Users\ASUS\PROJECT001\stamp_processing


c:\Users\ASUS\PROJECT001\stamp_processing\stamp_processing\module\unet.py:57: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if ssh != up_out.shape[-2:]:
c:\Users\ASUS\anaconda3_2\envs\stamp_processing\lib\site-packages\fastai\vision\models\unet.py:47: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.orig.shape[-2:] != x.shape[-2:]:
c:\Users\ASUS\anaconda3_2\envs\stamp_processing\lib\site-packages\torch\jit\_trace.py:152: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated du

### Convert Torch.Jit.Script model to ONNX

In [80]:
import torch.onnx
%cd C:/Users/ASUS/PROJECT001/stamp_processing

from PIL import Image
from fastai.vision.all import *

# unet_model = load_learner(r"C:\Users\ASUS\PROJECT001\stamp_processing\RedStampRemover.pt").model
unet_model = torch.jit.load(r"C:\Users\ASUS\PROJECT001\stamp_processing\RedStampRemover320x320.pt").cuda()

def Convert_ONNX(model): 
    unet_model.eval()
    
    dummy_input = torch.randn([1,3, 320, 320], requires_grad=True).cuda()
    dummy_output = unet_model(dummy_input)

    torch.onnx.export(model,         # model being run 
            dummy_input,       # model input (or a tuple for multiple inputs) 
            "RedStampRemover320x320.onnx",       # where to save the model  
            export_params=True,  # store the trained parameter weights inside the model file 
            opset_version=13,    # the ONNX version to export the model to 
            do_constant_folding=True,  # whether to execute constant folding for optimization 
            input_names = ['modelInput'],   # the model's input names 
            output_names = ['modelOutput'], # the model's output names
            dynamic_axes= {'modelInput' : {0: "BS"},    # variable length axes 
                        'modelOutput' : {0: "BS"}},
            example_outputs=dummy_output) 

    print('Model has been converted to ONNX') 

Convert_ONNX(unet_model)

C:\Users\ASUS\PROJECT001\stamp_processing
Model has been converted to ONNX


In [12]:
import onnx

onnx_model = onnx.load("RedStampRemover.onnx")
onnx.checker.check_model(onnx_model)

None


## Inference with one image

In [13]:
import onnxruntime
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
from fastai.vision.all import *


def imshow(img, figsize=(25, 25), **kwargs):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    ax.imshow(img, **kwargs)

def image_transform_onnx(image) -> np.ndarray:
    image = np.array(image)
    image = image.transpose(2,0,1).astype(np.float32)
    image /= 255
    image = image[None,...]
    return image
    

ort_session = onnxruntime.InferenceSession("RedStampRemover.onnx")
test_onnx_numpy = image_transform_onnx(Image.open(r"C:\Users\ASUS\PROJECT001\stamp_processing\sec1.jpg"))

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: test_onnx_numpy}
ort_outs = ort_session.run([ort_session.get_outputs()[0].name], ort_inputs)

def decode_prediction(preds):
    out = []
    i2f = IntToFloatTensor()
    for pred in preds:
        img_np = i2f.decodes(pred.squeeze())
        img_np = img_np.transpose(1, 2, 0)
        img_np *= 255
        img_np = img_np.astype(np.uint8)
        out.append(img_np)
        del img_np
        
    return out

imshow(decode_prediction(ort_outs[0])[0], figsize=(10, 10))

Fail: [ONNXRuntimeError] : 1 : FAIL : Non-zero status code returned while running Concat node. Name:'Concat_56' Status Message: concat.cc:159 onnxruntime::ConcatBase::PrepareForCompute Non concat axis dimensions must match: Axis 3 has mismatched dimensions of 107 and 108

## Inference ONNX model

In [5]:
%cd C:/Users/ASUS/PROJECT001/stamp_processing

from stamp_processing import StampRemoverONNX
import cv2

remover = StampRemoverONNX(removal_weight=r"C:\Users\ASUS\PROJECT001\stamp_processing\RedStampRemover.onnx")
preds = remover([cv2.imread("C:/Users/ASUS/PROJECT001/data/test-repo-master/sec/sec5.jpg")])

C:\Users\ASUS\PROJECT001\stamp_processing


IndexError: list index out of range